In [2]:
import pandas as pd

#load the datasets
raw = pd.read_csv('data/raw_data.csv')

incremental = pd.read_csv('data/incremental_data.csv')

In [3]:
# Transforming raw_data.csv

# Cleaning
# Before: check for duplicates
print("Duplicates in raw data:", raw.duplicated().sum())

# Drop duplicates
raw = raw.drop_duplicates()

# After: verify they're gone
print("Duplicates after cleaning:", raw.duplicated().sum())



Duplicates in raw data: 1
Duplicates after cleaning: 0


Removed duplicate rows to ensure consistency and prevent overcounting during analysis

In [4]:
# Handling missing values in raw_data

# Before
print("missing values before:",raw.isnull().sum())

# Categorical
raw['customer_name'] = raw['customer_name'].fillna('Unknown')
raw['region'] = raw['region'].fillna(raw['region'].mode()[0])

# Numeric (use median to reduce outlier distortion)
raw['quantity'] = raw['quantity'].fillna(raw['quantity'].median())
raw['unit_price'] = raw['unit_price'].fillna(raw['unit_price'].median())

# Dates: convert to datetime first
raw['order_date'] = pd.to_datetime(raw['order_date'], errors='coerce')
raw['order_date'] = raw['order_date'].ffill() #fillna(method='bfill')  

#after
raw.isnull().sum()

missing values before: order_id          0
customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64


order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64

Handled missing values across both categorical and numeric fields using context-appropriate strategies. Categorical entries were filled with the mode or a placeholder label. Numeric columns used the median to minimize distortion from skew or outliers.

In [5]:
# Structural transformation

# Before
print("before data type:", raw[['order_date']].dtypes)

# Convert 'order_date' to datetime
raw['order_date'] = pd.to_datetime(raw['order_date'], errors='coerce')

# After
print("After conversion:", raw['order_date'].dtypes)


before data type: order_date    datetime64[ns]
dtype: object
After conversion: datetime64[ns]


Converted the order_date column to datetime format to support time-based analysis and sorting.

In [6]:
# Enrichment

# Add total_price = quantity × unit_price
raw['total_price'] = raw['quantity'] * raw['unit_price']

#after
raw[['quantity', 'unit_price', 'total_price']].head()


,quantity,unit_price,total_price
0,2.0,500.0,1000.0
1,2.0,500.0,1000.0
2,2.0,250.0,500.0
3,2.0,750.0,1500.0
4,3.0,500.0,1500.0


Created a new total_price column by multiplying quantity and unit_price. This metric allows for future revenue aggregation and transactional value analysis.

In [21]:
# Categorization

# Define bins and labels
bins = [0-500, 600, 1000, float('inf')]
labels = ['Low', 'Medium', 'High']

# Create a new column
raw['order_value_category'] = pd.cut(raw['total_price'], bins=bins, labels=labels, right=False)

# Preview the result
raw[['total_price', 'order_value_category']].head()


,total_price,order_value_category
0,1000.0,High
1,1000.0,High
2,500.0,Low
3,1500.0,High
4,1500.0,High


Created a order_value_category column by binning total_price into tiers: Low (<100), Medium (100–499), and High (500+). This enables easier analysis of customer spending levels.

In [8]:
# Save transformed raw_data as transformed_full.csv
raw.to_csv('transformed/transformed_full.csv', index=False)


Transforming incremental_data.csv

In [9]:
#Cleaning: handle missing values

# Before 
print("Missing values:", incremental.isnull().sum())

# Work on the missing values
# Categorical
incremental['customer_name'] = incremental['customer_name'].fillna('Unknown')
incremental['region'] = incremental['region'].fillna(incremental['region'].mode()[0])

# Numeric (use median to reduce outlier distortion)
incremental['quantity'] = incremental['quantity'].fillna(incremental['quantity'].median())

#after
incremental.isnull().sum()



Missing values: order_id         0
customer_name    6
product          0
quantity         4
unit_price       0
order_date       0
region           2
dtype: int64


order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64

Handled missing values across both categorical and numeric fields using context-appropriate strategies. Categorical entries were filled with the mode or a placeholder label. Numeric columns used the median to minimize distortion from skew or outliers.

In [10]:
# Structural fixes

# Before
print("order_date dtype before:", incremental['order_date'].dtype)

# Convert 'order_date' to datetime
incremental['order_date'] = pd.to_datetime(incremental['order_date'], errors='coerce')

# After
print("After conversion:", incremental['order_date'].dtypes)



order_date dtype before: object
After conversion: datetime64[ns]


Converted the order_date column to datetime format to support time-based analysis and sorting.

In [11]:
# Enrichment

# Add total_price = quantity × unit_price
incremental['total_price'] = incremental['quantity'] * incremental['unit_price']

#after
print(incremental[['quantity','unit_price','total_price']].head())


   quantity  unit_price  total_price
0       1.5       900.0       1350.0
1       1.0       300.0        300.0
2       1.0       600.0        600.0
3       1.5       300.0        450.0
4       2.0       600.0       1200.0


Created a new total_price column by multiplying quantity and unit_price. This metric allows for future revenue aggregation and transactional value analysis.

In [20]:
# Define bins and labels
bins = [0-500, 600, 1000, float('inf')]
labels = ['Low', 'Medium', 'High']

# Create a new column
incremental['order_value_category'] = pd.cut(incremental['total_price'], bins=bins, labels=labels, right=False)

# Preview the result
incremental[['total_price', 'order_value_category']].tail()


,total_price,order_value_category
5,1200.0,High
6,600.0,Medium
7,900.0,Medium
8,1200.0,High
9,1350.0,High


Created a order_value_category column by binning total_price into tiers: Low (<100), Medium (100–499), and High (500+). This enables easier analysis of customer spending levels.

In [13]:
# Save transformed incremental_data as transformed_incremental.csv
incremental.to_csv('transformed/transformed_incremental.csv', index=False)
